In [ ]:
!pip install streamlit

In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import os
import sys

In [2]:
PROJECT_ID='diamonds-341821'
DATA_BUCKET ='gs://diamonds_data/diamonds.csv'
MODEL_STORAGE = os.path.join("gs://", 'stored_model', 'stored_model', 'v1')

In [4]:
!gcloud projects list

PROJECT_ID       NAME      PROJECT_NUMBER
diamonds-341821  diamonds  115343194111


In [5]:
!gcloud config set project $PROJECT_ID

Updated property [core/project].


In [6]:
##read the dataset from GS

df = pd.read_csv(DATA_BUCKET)

df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [7]:
df.pop('price')
df.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75


In [19]:
##checking the datatype
print(df['carat'].dtype)

float64


In [20]:
df.dtypes

carat      float64
cut         object
color       object
clarity     object
depth      float64
table      float64
price        int64
x          float64
y          float64
z          float64
dtype: object

In [50]:
for column in df:
    print("column {} is type: {}".format(column, df[column].dtype))

column carat is type: float64
column cut is type: object
column color is type: object
column clarity is type: object
column depth is type: float64
column table is type: float64
column x is type: float64
column y is type: float64
column z is type: float64


In [3]:
!pwd

/home/jupyter/diamonds


In [19]:
%%writefile UI/diamonds.py

import numpy as np
import pandas as pd
import streamlit as st
import json
import requests

DATA_BUCKET ='gs://diamonds_data/diamonds.csv'

url_serving = 'https://diamonds-image-bfpumxj2xa-uc.a.run.app/v1/models/diamonds:predict'

st.title("Diamond price prediction tool")

st.image('diamond-434.gif')

st.subheader("Please, select your diamond characteristics:")

##load the diamond dataframe


df = pd.read_csv(DATA_BUCKET)
df.pop('price')

##selection.append(st.slider(column, min_value=float(red_df[column].min()),max_value=float(red_df[column].max()),value=float(red_df[column].mean()),format='%2f'))

def rest_request(text, url=None):
    
    if url is None:
        url='https://diamonds-image-bfpumxj2xa-uc.a.run.app/v1/models/diamonds:predict'
        
    payload = json.dumps({"instances":[text]})
    print(payload)
    response = requests.post(url,data=payload)
        
    return response
    

diamond_desc = {
    'carat':'Weight of the diamond',
    'cut': "Quality of the cut",
    'color':'Diamond color',
    'clarity':'Clarity of the diamond',
    'x':'Length in mm',
    'y':'Width in mm',
    'z': 'Depth in mm',
    'depth':'Total depth percentage',
    'table':'Width of top of the diamond relative to widest point'
}

diamond_values = {
    'cut':['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'],
    'color':['D(best)','E','F','G','H','I','J(worst)'],
    'clarity':['I1 (worst)', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF (best)']  
}

NUMERIC_COLS = ['carat','x','y','z','depth','table']

CAT_COLS = ['cut','color','clarity']

selection = {}

for column in df:
    
    if column in NUMERIC_COLS:
        
        selection[column] = [st.slider(diamond_desc[column], min_value=float(df[column].min()),max_value=float(df[column].max()),value=float(round(df[column].mean(),2)))]

    if column in CAT_COLS:
        
        selection[column] = [st.selectbox(diamond_desc[column],diamond_values[column])]
        
        
##st.text(selection)

if st.button("Predict the price!"):
    rs = rest_request(selection, url_serving)
    st.subheader("The estimated price of the diamond is ${0:,}".format(int(rs.json()['predictions'][0][0]), unsafe_allow_html=True))

##rs = rest_request(selection)
##st.write(rs.json()['predictions'][0][0])

Overwriting UI/diamonds.py


In [15]:
%%writefile UI/Dockerfile

FROM python:3.8
RUN pip install -U streamlit numpy pandas gcsfs
WORKDIR /app
COPY diamonds.py diamond-434.gif ./
EXPOSE 8501
ENTRYPOINT ["streamlit", "run"]
CMD ["diamonds.py"]

Overwriting UI/Dockerfile


In [10]:
IMAGE_NAME='diamonds_ui_image'
IMAGE_TAG='latest'
IMAGE_URI='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, IMAGE_TAG)

In [20]:
!gcloud builds submit UI --tag $IMAGE_URI

Creating temporary tarball archive of 3 file(s) totalling 9.9 MiB before compression.
Uploading tarball of [UI] to [gs://diamonds-341821_cloudbuild/source/1646334738.981767-839fbad80fe047ca98299289302e9a05.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/diamonds-341821/locations/global/builds/50854451-247e-4a8e-8995-8fe793f231e7].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/50854451-247e-4a8e-8995-8fe793f231e7?project=115343194111].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "50854451-247e-4a8e-8995-8fe793f231e7"

FETCHSOURCE
Fetching storage object: gs://diamonds-341821_cloudbuild/source/1646334738.981767-839fbad80fe047ca98299289302e9a05.tgz#1646334740353041
Copying gs://diamonds-341821_cloudbuild/source/1646334738.981767-839fbad80fe047ca98299289302e9a05.tgz#1646334740353041...
/ [1 files][  9.9 MiB/  9.9 MiB]                                                
Operation completed over 1 object